# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [3]:
import math
def InitialDataEqualMassConic(ecc,angle,initmass):
    orbitalangle=angle
    phi=np.array([orbitalangle,math.pi+orbitalangle])
    orbitalradius=100*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [4]:
import random,numpy as np
initdateqellipse=InitialDataEqualMassConic(0.1,0.,1.)
print(initdateqellipse)

0.1
(array([0.        , 3.14159265]), 0.0, array([100., 100.]), 0.1, array([1., 1.]))


In [15]:
def getxyuveqconicredmass(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius[0]/2. #reduced mass
        E=Gconstant*masses[0]*masses[1]/orbitalr #orbitalradius is correct here r1+r2
        coordsep=orbitalr*(1.+eccentricity) #starts at apastron
        starsep=coordsep #reduced mass, M is at center
        Eapastron=Gconstant*masses[0]*masses[1]/(starsep)
        Fapastron=Eapastron/(starsep)
        print("E,F",E,Fapastron,2*orbitalr,starsep)
        x0=coordsep*cosphi
        y0=coordsep*sinphi
        v=np.zeros(2)
        v=np.sqrt(2.*(2.*Eapastron-E)/masses) #virial theorem (not the viral theorem, the VEE-REE-AHL theoerem, a thermodynamics theorem about stellar dynamics and energy among other things)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        for i in np.arange(2):
            a[i]=Gconstant*masses[(i+1)%2]/(r0[i]**2)
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [16]:
xyuvaeqconic=getxyuveqconic(initdateqellipse)
print(xyuvaeqconic)

initdat [100. 100.] [0.         3.14159265] [ 1. -1.] [0.0000000e+00 1.2246468e-16]
coordsep [100. 100.]
zero
pi
[ 100. -100.]
[0. 0.]
0.1
E,F 0.005 2.0661157024793383e-05 200.0 220.00000000000003
(array([1., 1.]), array([ 110., -110.]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([ 0.0904534, -0.0904534]), array([0., 0.]), array([-2.0661157e-05,  2.0661157e-05]), array([-0., -0.]), array([0., 0.]))


In [17]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                #print(rreljk)
                axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [18]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqconic
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 110. -110.]
[ 110. -110.] [ 0.0904534 -0.0904534] [-2.0661157e-05  2.0661157e-05] [ 0.0904534 -0.0904534] [-2.0661157e-05  2.0661157e-05] [0. 0.]
[ 109.99997934 -109.99997934] [ 0.18090681 -0.18090681] [-4.1322314e-05  4.1322314e-05] [ 0.0904534 -0.0904534] [-2.06611361e-05  2.06611361e-05] [-1.6989728e-08  1.6989728e-08]
[ 109.99993802 -109.99993802] [ 0.27136021 -0.27136021] [-6.19834501e-05  6.19834501e-05] [ 0.09045339 -0.09045339] [-2.0661081e-05  2.0661081e-05] [-3.39793717e-08  3.39793717e-08]
[ 109.99987603 -109.99987603] [ 0.3618136 -0.3618136] [-8.26445311e-05  8.26445311e-05] [ 0.09045335 -0.09045335] [-2.06609917e-05  2.06609917e-05] [-5.09688565e-08  5.09688565e-08]
[ 109.99979339 -109.99979339] [ 0.45226695 -0.45226695] [-0.00010331  0.00010331] [ 0.0904533 -0.0904533] [-2.06608683e-05  2.06608683e-05] [-6.79581045e-08  6.79581045e-08]
[ 109.99969008 -109.99969008] [ 0.54272025 -0.54272025] [-0.00012397  0.00012397] [ 0.09045323 -0.09045323] [-2.06607107e-05  2.06607107

[ 109.59912935 -109.59912935] [ 17.88863932 -17.88863932] [-0.00404829  0.00404829] [ 0.09013016 -0.09013016] [-2.00141677e-05  2.00141677e-05] [-3.25010967e-06  3.25010967e-06]
[ 109.59508106 -109.59508106] [ 17.97876948 -17.97876948] [-0.00406831  0.00406831] [ 0.09012691 -0.09012691] [-2.00077516e-05  2.00077516e-05] [-3.26564139e-06  3.26564139e-06]
[ 109.59101275 -109.59101275] [ 18.06889639 -18.06889639] [-0.00408832  0.00408832] [ 0.09012365 -0.09012365] [-2.00013064e-05  2.00013064e-05] [-3.28115891e-06  3.28115891e-06]
[ 109.58692444 -109.58692444] [ 18.15902004 -18.15902004] [-0.00410832  0.00410832] [ 0.09012037 -0.09012037] [-1.9994832e-05  1.9994832e-05] [-3.29666218e-06  3.29666218e-06]
[ 109.58281612 -109.58281612] [ 18.24914041 -18.24914041] [-0.00412831  0.00412831] [ 0.09011707 -0.09011707] [-1.99883284e-05  1.99883284e-05] [-3.31215114e-06  3.31215114e-06]
[ 109.57868781 -109.57868781] [ 18.33925747 -18.33925747] [-0.0041483  0.0041483] [ 0.09011376 -0.09011376] [-1.

[ 109.10834716 -109.10834716] [ 26.70364026 -26.70364026] [-0.00597493  0.00597493] [ 0.08974019 -0.08974019] [-1.92544247e-05  1.92544247e-05] [-4.69631598e-06  4.69631598e-06]
[ 109.10237223 -109.10237223] [ 26.79338045 -26.79338045] [-0.00599418  0.00599418] [ 0.0897355 -0.0897355] [-1.92453969e-05  1.92453969e-05] [-4.71020018e-06  4.71020018e-06]
[ 109.09637805 -109.09637805] [ 26.88311594 -26.88311594] [-0.00601343  0.00601343] [ 0.08973078 -0.08973078] [-1.92363453e-05  1.92363453e-05] [-4.72406519e-06  4.72406519e-06]
[ 109.09036462 -109.09036462] [ 26.97284673 -26.97284673] [-0.00603266  0.00603266] [ 0.08972606 -0.08972606] [-1.922727e-05  1.922727e-05] [-4.73791099e-06  4.73791099e-06]
[ 109.08433195 -109.08433195] [ 27.06257279 -27.06257279] [-0.00605189  0.00605189] [ 0.08972132 -0.08972132] [-1.9218171e-05  1.9218171e-05] [-4.75173753e-06  4.75173753e-06]
[ 109.07828006 -109.07828006] [ 27.15229411 -27.15229411] [-0.00607111  0.00607111] [ 0.08971657 -0.08971657] [-1.9209

[ 107.93186519 -107.93186519] [ 40.8158944 -40.8158944] [-0.00889138  0.00889138] [ 0.08883921 -0.08883921] [-1.75737086e-05  1.75737086e-05] [-6.63072433e-06  6.63072433e-06]
[ 107.92297382 -107.92297382] [ 40.90473361 -40.90473361] [-0.00890895  0.00890895] [ 0.08883258 -0.08883258] [-1.75617155e-05  1.75617155e-05] [-6.64120022e-06  6.64120022e-06]
[ 107.91406487 -107.91406487] [ 40.99356619 -40.99356619] [-0.00892651  0.00892651] [ 0.08882594 -0.08882594] [-1.7549709e-05  1.7549709e-05] [-6.65165297e-06  6.65165297e-06]
[ 107.90513836 -107.90513836] [ 41.08239213 -41.08239213] [-0.00894406  0.00894406] [ 0.08881928 -0.08881928] [-1.75376889e-05  1.75376889e-05] [-6.66208257e-06  6.66208257e-06]
[ 107.8961943 -107.8961943] [ 41.17121141 -41.17121141] [-0.0089616  0.0089616] [ 0.08881262 -0.08881262] [-1.75256555e-05  1.75256555e-05] [-6.67248902e-06  6.67248902e-06]
[ 107.8872327 -107.8872327] [ 41.26002403 -41.26002403] [-0.00897912  0.00897912] [ 0.08880595 -0.08880595] [-1.751360

[ 106.75378588 -106.75378588] [ 51.33852217 -51.33852217] [-0.01089526  0.01089526] [ 0.08798298 -0.08798298] [-1.60693687e-05  1.60693687e-05] [-7.7138226e-06  7.7138226e-06]
[ 106.74289062 -106.74289062] [ 51.42650515 -51.42650515] [-0.01091133  0.01091133] [ 0.08797526 -0.08797526] [-1.60562178e-05  1.60562178e-05] [-7.72152936e-06  7.72152936e-06]
[ 106.7319793 -106.7319793] [ 51.51448041 -51.51448041] [-0.01092738  0.01092738] [ 0.08796754 -0.08796754] [-1.60430608e-05  1.60430608e-05] [-7.72921311e-06  7.72921311e-06]
[ 106.72105191 -106.72105191] [ 51.60244795 -51.60244795] [-0.01094343  0.01094343] [ 0.08795981 -0.08795981] [-1.60298979e-05  1.60298979e-05] [-7.73687386e-06  7.73687386e-06]
[ 106.71010848 -106.71010848] [ 51.69040777 -51.69040777] [-0.01095946  0.01095946] [ 0.08795207 -0.08795207] [-1.6016729e-05  1.6016729e-05] [-7.74451162e-06  7.74451162e-06]
[ 106.69914903 -106.69914903] [ 51.77835984 -51.77835984] [-0.01097547  0.01097547] [ 0.08794433 -0.08794433] [-1.60

...

[-102.38240086  102.38240086] [ 653.83527489 -653.83527489] [-0.02484878  0.02484878] [ 0.06145305 -0.06145305] [ 8.83083014e-08 -8.83083014e-08] [-5.64039043e-07  5.64039043e-07]
[-102.40724965  102.40724965] [ 653.89672794 -653.89672794] [-0.0248487  0.0248487] [ 0.06145249 -0.06145249] [ 8.83038928e-08 -8.83038928e-08] [-5.63926999e-07  5.63926999e-07]
[-102.43209834  102.43209834] [ 653.95818043 -653.95818043] [-0.02484861  0.02484861] [ 0.06145192 -0.06145192] [ 8.82994818e-08 -8.82994818e-08] [-5.63814988e-07  5.63814988e-07]
[-102.45694695  102.45694695] [ 654.01963236 -654.01963236] [-0.02484852  0.02484852] [ 0.06145136 -0.06145136] [ 8.82950684e-08 -8.82950684e-08] [-5.63703011e-07  5.63703011e-07]
[-102.48179547  102.48179547] [ 654.08108372 -654.08108372] [-0.02484843  0.02484843] [ 0.0614508 -0.0614508] [ 8.82906525e-08 -8.82906525e-08] [-5.63591067e-07  5.63591067e-07]
[-102.5066439  102.5066439] [ 654.14253452 -654.14253452] [-0.02484834  0.02484834] [ 0.06145023 -0.0614

[-106.43159004  106.43159004] [ 663.84475389 -663.84475389] [-0.02483445  0.02483445] [ 0.06136257 -0.06136257] [ 8.75591057e-08 -8.75591057e-08] [-5.46208554e-07  5.46208554e-07]
[-106.45642449  106.45642449] [ 663.90611646 -663.90611646] [-0.02483436  0.02483436] [ 0.06136202 -0.06136202] [ 8.75543296e-08 -8.75543296e-08] [-5.46101795e-07  5.46101795e-07]
[-106.48125885  106.48125885] [ 663.96747848 -663.96747848] [-0.02483428  0.02483428] [ 0.06136148 -0.06136148] [ 8.75495514e-08 -8.75495514e-08] [-5.45995068e-07  5.45995068e-07]
[-106.50609313  106.50609313] [ 664.02883996 -664.02883996] [-0.02483419  0.02483419] [ 0.06136093 -0.06136093] [ 8.75447712e-08 -8.75447712e-08] [-5.45888371e-07  5.45888371e-07]
[-106.53092731  106.53092731] [ 664.09020089 -664.09020089] [-0.0248341  0.0248341] [ 0.06136039 -0.06136039] [ 8.75399888e-08 -8.75399888e-08] [-5.45781706e-07  5.45781706e-07]
[-106.55576141  106.55576141] [ 664.15156128 -664.15156128] [-0.02483401  0.02483401] [ 0.06135984 -0.

[-109.63451294  109.63451294] [ 671.7560529 -671.7560529] [-0.0248232  0.0248232] [ 0.06129298 -0.06129298] [ 8.69266253e-08 -8.69266253e-08] [-5.32691579e-07  5.32691579e-07]
[-109.65933614  109.65933614] [ 671.81734588 -671.81734588] [-0.02482311  0.02482311] [ 0.06129245 -0.06129245] [ 8.69215992e-08 -8.69215992e-08] [-5.3258877e-07  5.3258877e-07]
[-109.68415925  109.68415925] [ 671.87863832 -671.87863832] [-0.02482302  0.02482302] [ 0.06129191 -0.06129191] [ 8.69165714e-08 -8.69165714e-08] [-5.32485991e-07  5.32485991e-07]
[-109.70898227  109.70898227] [ 671.93993024 -671.93993024] [-0.02482293  0.02482293] [ 0.06129138 -0.06129138] [ 8.69115417e-08 -8.69115417e-08] [-5.32383242e-07  5.32383242e-07]
[-109.7338052  109.7338052] [ 672.00122162 -672.00122162] [-0.02482285  0.02482285] [ 0.06129085 -0.06129085] [ 8.69065102e-08 -8.69065102e-08] [-5.32280523e-07  5.32280523e-07]
[-109.75862805  109.75862805] [ 672.06251247 -672.06251247] [-0.02482276  0.02482276] [ 0.06129032 -0.061290

[-113.03448304  113.03448304] [ 680.14827125 -680.14827125] [-0.02481133  0.02481133] [ 0.06122095 -0.06122095] [ 8.62220174e-08 -8.62220174e-08] [-5.18880169e-07  5.18880169e-07]
[-113.05929437  113.05929437] [ 680.20949219 -680.20949219] [-0.02481125  0.02481125] [ 0.06122043 -0.06122043] [ 8.62167613e-08 -8.62167613e-08] [-5.18781345e-07  5.18781345e-07]
[-113.08410562  113.08410562] [ 680.27071262 -680.27071262] [-0.02481116  0.02481116] [ 0.06121991 -0.06121991] [ 8.62115036e-08 -8.62115036e-08] [-5.1868255e-07  5.1868255e-07]
[-113.10891678  113.10891678] [ 680.33193253 -680.33193253] [-0.02481108  0.02481108] [ 0.06121939 -0.06121939] [ 8.62062444e-08 -8.62062444e-08] [-5.18583783e-07  5.18583783e-07]
[-113.13372785  113.13372785] [ 680.39315192 -680.39315192] [-0.02481099  0.02481099] [ 0.06121887 -0.06121887] [ 8.62009836e-08 -8.62009836e-08] [-5.18485045e-07  5.18485045e-07]
[-113.15853884  113.15853884] [ 680.45437079 -680.45437079] [-0.0248109  0.0248109] [ 0.06121835 -0.06

[-116.85441586  116.85441586] [ 689.57024242 -689.57024242] [-0.02479812  0.02479812] [ 0.06114219 -0.06114219] [ 8.53952709e-08 -8.53952709e-08] [-5.03988779e-07  5.03988779e-07]
[-116.87921398  116.87921398] [ 689.63138461 -689.63138461] [-0.02479803  0.02479803] [ 0.06114168 -0.06114168] [ 8.53897951e-08 -8.53897951e-08] [-5.03894194e-07  5.03894194e-07]
[-116.90401202  116.90401202] [ 689.69252629 -689.69252629] [-0.02479795  0.02479795] [ 0.06114118 -0.06114118] [ 8.5384318e-08 -8.5384318e-08] [-5.03799636e-07  5.03799636e-07]
[-116.92880996  116.92880996] [ 689.75366747 -689.75366747] [-0.02479786  0.02479786] [ 0.06114067 -0.06114067] [ 8.53788396e-08 -8.53788396e-08] [-5.03705104e-07  5.03705104e-07]
[-116.95360783  116.95360783] [ 689.81480814 -689.81480814] [-0.02479778  0.02479778] [ 0.06114017 -0.06114017] [ 8.53733599e-08 -8.53733599e-08] [-5.036106e-07  5.036106e-07]
[-116.9784056  116.9784056] [ 689.87594831 -689.87594831] [-0.02479769  0.02479769] [ 0.06113967 -0.061139

In [19]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Equal Mass System, Ellipse", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="Star 1")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="Star 2")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

hyperbolic. energy is too high